In [5]:
import json
import openai
import json
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


In [ ]:

# OpenAI API 키 설정
openai_api_key = 'MY_OPEN_AI_KEY'

# OpenAI Chat 모델 설정
llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4-turbo")

# 프롬프트 템플릿 설정
system_template = SystemMessagePromptTemplate.from_template(
    "You are an expert in agricultural issues."
)
human_template = HumanMessagePromptTemplate.from_template(
    "다음은 '{title}'안에 있는 질문입니다. '{input_text}' 에 대한 딸기 농업 전문가의 답변을 생성해 주세요"
)
chat_prompt = ChatPromptTemplate.from_messages([system_template, human_template])

# LLMChain 생성
chain = LLMChain(llm=llm, prompt=chat_prompt)

# expert_data.json 파일을 업데이트하는 함수
def update_expert_data():
    # 기존 JSON 데이터를 불러옵니다.
    with open('./make_eval/eval_data.json', 'r', encoding='utf-8') as f:
        queries = json.load(f)
    
    # 각 항목에 대해 GPT-4 Turbo 응답을 생성하고 추가
    for query in queries:
        title = query['title']
        input_text = query['input_text']
        base_model_response = chain.run({"title": title, "input_text": input_text})
        query['base_model_response'] = base_model_response

    # 업데이트된 JSON 데이터를 파일에 다시 저장
    with open('./make_eval/eval_data.json','w',encoding='utf-8') as f:
        json.dump(queries, f, ensure_ascii=False, indent=4)

# 업데이트 함수 실행
update_expert_data()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings


openai_api_key = 'MY_API_KEY'


embeddings = OpenAIEmbeddings(api_key=openai_api_key)
knowledge_base_path = "../index_store2"
vector_index = Chroma(persist_directory=knowledge_base_path, embedding_function=embeddings)
retriever = vector_index.as_retriever(search_type = "similarity",
                                      search_kwargw={"k": 3}
                                      )
llm = ChatOpenAI(temperature = 0,
                 model_name="gpt-4-turbo",
                 openai_api_key = openai_api_key)
chain = RetrievalQA.from_llm(
    llm = llm,
    retriever = retriever
)

def update_expert_data():
    # 기존 JSON 데이터를 불러옵니다.
    with open('./make_eval/eval_data.json', 'r', encoding='utf-8') as f:
        queries = json.load(f)
    
    # 각 항목에 대해 GPT-4 Turbo 응답을 생성하고 추가
    for query in queries:
        title = query['title']
        input_text = query['input_text']
        user_question = title + input_text
        base_model_response = chain.run({"query": user_question})
        query['RAG_model_response'] = base_model_response

    # 업데이트된 JSON 데이터를 파일에 다시 저장
    with open('./make_eval/eval_data.json','w',encoding='utf-8') as f:
        json.dump(queries, f, ensure_ascii=False, indent=4)

# 업데이트 함수 실행
update_expert_data()

In [ ]:
import os
import json
import shutil
from PyPDF2 import PdfReader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# OpenAI API 키 설정
openai_api_key = 'MY_API_KEY'

# 임베딩 설정
embeddings = OpenAIEmbeddings(api_key=openai_api_key)

# Chroma 데이터베이스 초기화
knowledge_base_path = "./index_store"
if os.path.exists(knowledge_base_path):
    try:
        shutil.rmtree(knowledge_base_path)
        print(f"{knowledge_base_path} 디렉토리가 성공적으로 삭제되었습니다.")
    except Exception as e:
        print(f"{knowledge_base_path} 디렉토리 삭제에 실패했습니다. 이유: {e}")

os.makedirs(knowledge_base_path, exist_ok=True)

vector_index = Chroma(embedding_function=embeddings, persist_directory=knowledge_base_path)

# 폴더 경로 설정
folder_path = "../pdf"

def read_pdf(file_path):
    content = ""
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            content += page.extract_text()
    return content

def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def process_files(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith('.pdf'):
                content = read_pdf(file_path)
                vector_index.add_texts([content], [{"source": file_path}])
            elif file.endswith('.json'):
                content = read_json(file_path)
                vector_index.add_texts([json.dumps(content)], [{"source": file_path}])

process_files(folder_path)

# 데이터베이스 저장
vector_index.persist()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(
